## Evaluating the three-body fermion bound state

Here we will generate plots by evaluating the three-body fermion bound state.

The following plots will be evaluated:
- wavefunction structure and its dependence on q wrt different ang mom states (J=1/2, 3/2)
    - also compare binding energy between the two
- comparison between boson and fermion case for wavefunction and binding energy for specific angular mom state

### Notes about plotting
- do it like Dr. Nogga's paper 
- plot 3-d plot of |wavefunc| vs p and q (p12 and p3)
- plot 2-d contour plot of |wavefunc| vs p and q
- evaluate KE (and possibly other operators too) for different cases
- plot for off-shell t-matrix (?)

In [1]:
from tbs_cls import OBEpot, ThreeBody
from tfbs_cls import TwoBodyTMatFerm, ThreeBodyFerm

import numpy as np
import matplotlib.pyplot as plt
import math as m


In [2]:
# parameters for OBE interaction
parasets=[[300.0, -0.09827953494014054],
 [400.0, -0.028203145146196713],
 [500.0, -0.0004221894040945335],
 [600.0, 0.012857431330421717],
 [700.0, 0.020167185806378923],
 [800.0, 0.024707945457255083],
 [900.0, 0.027865200396659445],
 [1000.0, 0.030308007813785776],
 [1100.0, 0.03239034331482156],
 [1200.0, 0.03431611357447293]]

para=[700.0, 0.020167185806378923]

### Comparisons between angular momentum states

- we vary bj for the two angular momentum states


In [3]:
bj_list = [0.5, 1.5]  # J=1/2, J=3/2
wf_list = []

pot=OBEpot(nx=24,mpi=138.0,C0=para[1],A=-1.0/6.474860194946856,cutoff=para[0])


for bj_idx, bj in enumerate(bj_list):
    # perform wavefunction evaluation for each angular momentum state
    kernel=ThreeBodyFerm(pot,nx=8,np1=20,np2=12,nq1=20,nq2=12,lmax=0, l3max=0, bj=bj)
    print(bj)
    ener,eta,fadcomp=kernel.esearch(neigv=1,e1=-0.0046,e2=-0.0048,elow=-0.001,tol=1e-5)
    wf = kernel.wavefunc(fadcomp)

    wf_list.append(wf)

evalue evaluation:  89.83453890000001
norm evaluation:  0.35068699999999353
evalue evaluation:  119.55329939999999
norm evaluation:  0.35238480000003847
evalue evaluation:  157.1409011
norm evaluation:  0.3766759000000093
evalue evaluation:  151.81325469999996
norm evaluation:  0.3746858000000657
evalue evaluation:  151.13498500000003
norm evaluation:  0.3887752000000546
evalue evaluation:  148.56660110000007
norm evaluation:  0.3501916000000165
evalue evaluation:  101.26011999999992
norm evaluation:  0.36781440000004295
evalue evaluation:  102.14379719999988
norm evaluation:  1.262146699999903
evalue evaluation:  102.68157389999988
norm evaluation:  1.0207313000000795
evalue evaluation:  101.24231170000007
norm evaluation:  0.9653525999999601


c:\Users\voltk\Projects\UBonn_WS2122\comp_phys\CompPhysWS2122\final_project\notebooks\tfbs_cls.py:731: RuntimeWarning: invalid value encountered in double_scalars
  enew=e2+(e1-e2)/(eta1-eta2)*(1-eta2)


evalue evaluation:  97.73730509999996
norm evaluation:  1.7779840999999124
evalue evaluation:  99.43627809999998
norm evaluation:  0.8803020000000288
evalue evaluation:  106.14941379999982
norm evaluation:  1.0062184000000798


KeyboardInterrupt: 

In [ ]:
# momentum grids are the same for each kernel, so can just use any of them
pgrid = kernel.pgrid
qgrid = kernel.qgrid

# create combined grid
P, Q = np.meshgrid(pgrid, qgrid)

# plotting the results
fig, ax = plt.subplots(figsize=(6,4))


for bj_idx, bj in enumerate(bj_list):
    ax.contourf()

In [5]:
-0.004713309405096219 - (-0.00471297673679547)

-3.3266830074939885e-07